In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
##Load the dataset
data = pd.read_csv('Churn_Modelling.csv')

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
##Preprocessing the data, by removing the columns that are not needed

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
##handling categorical data, geography and gender

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender']) ## converting gender to numerical values, fit_transfrom is used to transform the data
data 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
##geography is also a categorical data, so we need to convert it to numerical values, but it has many categories
## we will not use label encoder, because it will assign unique values, and ANN might consider 1 category as more important than others
##so we use one hot encoding

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geography = OneHotEncoder()
geo_encoded = one_hot_encoder_geography.fit_transform(data[['Geography']])
geo_encoded 


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [15]:
geo_encoded.toarray()## converting the encoded data to an array, so that we can convert it to a dataframe

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [16]:
## we can see the features that got converted to numerical values
one_hot_encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [18]:
##converting the encoded data to a dataframe

geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df




,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [19]:
##Combine one hot encoded data with the original data

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [20]:
## save the encoders and scaler
## pickle is a type of file that can store python objects, so that we can use it later
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geography, file)



In [21]:
## divide the data set into independent and dependent variables

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [22]:
##exit is our dependent variable, and all other columns are independent variables

X= data.drop('Exited', axis=1)
y= data['Exited']

##split the data into training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## scale the data

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)## difference between transform and fit_transform is that fit_transform hides the mean and standard deviation, and transform uses the mean and standard deviation that is fitted
X_test = scaler.transform(X_test)





In [23]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)



ANN Implementation

In [24]:
##sequential model means, forward propagation and backward propagation
## we use dense to create hidden layers 
## activation function like sigmoid, relu, tanh, softmax, etc
##Optimizer used for backpropogation, for updating weights and biases
##Loss function used to reduce the error between predicted and actual values
##metrics as accuracy, precision, recall, f1 score, etc
##training, we use logs, tensorboard to monitor the training process
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [25]:
##Building the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )),##HL1, conenct to input layer.we only give input shape first layer, this ia a hideen layer
    Dense(32, activation='relu'),##HL2
    Dense(1, activation='sigmoid')##outpout
])

2024-12-25 00:37:24.457342: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-12-25 00:37:24.457377: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-12-25 00:37:24.457389: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-12-25 00:37:24.457436: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-25 00:37:24.457470: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [28]:
##compile model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy']) ##can initialize parameters anyway

In [41]:
##set up  tensorboard for training
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "log/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
##set up early stopping. we use this, when we are running multiple epochs are there isn't much difference in epochs. then we can stop the epochs if no difference

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)##atleast 5 epochs and consider best weights

In [43]:
##train model

history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),epochs=100,
     callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6442 - accuracy: 0.7952 - val_loss: 0.4342 - val_accuracy: 0.8170
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4517 - accuracy: 0.8086 - val_loss: 0.4236 - val_accuracy: 0.8185
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4671 - accuracy: 0.8044 - val_loss: 0.5175 - val_accuracy: 0.7815
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5237 - accuracy: 0.7949 - val_loss: 0.4487 - val_accuracy: 0.8045
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5575 - accuracy: 0.7855 - val_loss: 0.6747 - val_accuracy: 0.7870
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6953 - accuracy: 0.7710 - val_loss: 0.8026 - val_accuracy: 0.7815
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.9081 - accuracy: 0.7546 - val_loss: 0.7537 - val_accuracy: 0.7980

In [44]:
model.save('model.h5')

/Users/sriramsohan/Documents/projects/testing_models/pipeline_inf_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
##load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 60928), started 0:07:26 ago. (Use '!kill 60928' to kill it.)

In [ ]:
##Load the pickle files to make prediction
